In [1]:
import torch
import numpy as np

from model import get_model
from data import get_dataloaders

from config import batch_size, device
from loss_metrics import dice_pytorch, iou_pytorch

In [3]:
model = get_model(model_name="unet", encoder_name="vgg11")
checkpoint = torch.load(
    "outputs/no-transforms/unet-vgg11-B32-E50-LR0.001-20231121-150917/model.pth"
)
model.load_state_dict(checkpoint["model_state_dict"])

_, _, test_loader = get_dataloaders(batch_size)

/home/santhise001/miniforge3/envs/lgg-segmentation/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/santhise001/miniforge3/envs/lgg-segmentation/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/santhise001/miniforge3/envs/lgg-segmentation/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/santhise001/miniforge3/envs/lgg-segmentation/lib/python3.11/site-packages/sklearn/utils/valid

In [4]:
history = {"test_IoU": [], "test_dice": []}

# for i, (image, mask) in enumerate(test_loader):
# print(image.shape, mask.shape)
with torch.no_grad():
    running_IoU = 0
    running_dice = 0
    for i, data in enumerate(test_loader):
        # print(f"Processing for batch {i} / {len(test_loader)}")
        img, mask = data
        img, mask = img.to(device), mask.int().to(device)
        predictions = model(img)
        predictions = predictions.squeeze(1)
        history["test_IoU"].append(dice_pytorch(predictions, mask).cpu().numpy())
        history["test_dice"].append(iou_pytorch(predictions, mask).cpu().numpy())

print(f"Test IoU: {np.concatenate(history['test_IoU']).mean()}")
print(f"Test DICE: {np.concatenate(history['test_dice']).mean()}")

Test IoU: 0.9487584233283997
Test DICE: 0.9201500415802002


In [27]:
checkpoint["history"]["val_loss"]

[0.029669194188663515,
 0.030216932934472116,
 0.030065436143491227,
 0.030252569964376547,
 0.02980635720541922,
 0.030138420218885954,
 0.02979924888428995,
 0.030015859215409828]

In [32]:
import plotly.express as px
import pandas as pd

In [36]:
losses

,train_loss,val_loss,val_IoU,val_dice,num_epochs_trained
0,0.026422,0.029669,0.014076,0.026476,8
1,0.026424,0.030217,0.014374,0.027014,8
2,0.026236,0.030065,0.014211,0.026721,8
3,0.026234,0.030253,0.014332,0.026937,8
4,0.026179,0.029806,0.014182,0.026666,8
5,0.026342,0.030138,0.014308,0.026895,8
6,0.026433,0.029799,0.014118,0.026556,8
7,0.026123,0.030016,0.014259,0.026802,8


In [39]:
df = pd.DataFrame(checkpoint["history"]).drop("num_epochs_trained", axis=1)

In [40]:
px.line(df[["train_loss", "val_loss"]])